**<font color = blue size=5>網易雲音樂 CloudMusic</font>**<br>
https://music.163.com/

**<font color = blue size=4>教學</font>**<br>
https://www.jianshu.com/p/5ebd240bd7c2<br>
https://segmentfault.com/a/1190000014948845<br>
https://juejin.im/post/5ae33f96518825670d72d878<br>
https://kknews.cc/zh-tw/comic/o26onj6.html

1. 反爬蟲機制解決<br>
   https://www.jianshu.com/p/008c99877f30
2. 在網址列輸入"about:version"<br>
   https://blog.csdn.net/Pop_Rain/article/details/73178223<br>
   使用者代理程式	Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36
3. F12檢視<br>
   按F12 → Network → 選擇Doc → 選擇Headers → 查詢user-agent

In [1]:
import os
import re
import json
import urllib
import requests
import numpy as np
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# 收集歌單內的歌曲ID
# 取得歌曲頁面url，存成list(song_url)
# https://segmentfault.com/a/1190000014948845
# 測試歌單：https://music.163.com/#/playlist?id=2781412735

def get_songs(playlist_id):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    r = requests.get('http://music.163.com/playlist?id={}'.format(playlist_id),headers=headers)
    song_ids = re.findall(r'song\?id=(\d+?)".+?</a>',r.text) #歌曲id列表
    song_titles = re.findall(r'song\?id=\d+?">(.+?)</a>',r.text)#歌名title列表
    song_url = []
    for i in range(len(song_ids)):
        song_id = song_ids[i]
        url = 'https://music.163.com/#/song?id={}'.format(song_id)
        song_url.append(url)
        
    list_title = re.search(r'>(.+?) - 歌单 - 网易云音乐',r.text).group(1) #歌單名稱
    list_url = 'http://music.163.com/playlist?id=' + str(playlist_id) #歌單連結
    
    return [song_ids, song_titles, song_url, list_title, list_url]

In [4]:
# 歌單的歌轉成dataframe並存成csv
# 亂碼使用encoding='utf_8_sig'解決
def get_songinfo(song_ids, song_titles, song_url):
    df_songs = pd.DataFrame()
    df_songs['SongID'] = pd.Series(song_ids)
    df_songs['SongTitle'] = pd.Series(song_titles)
    df_songs['SongURL'] = pd.Series(song_url)
    df_songs.to_csv('CloudMusic_csv/df_songs.csv',encoding='utf_8_sig')
    return df_songs

In [5]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, " ", title)  # 替換為空白
    return new_title

In [6]:
# 取得歌名和歌手
def get_name_artist(df_songs, count):
    songname = df_songs['SongTitle'][count]
    songname = validateTitle(songname)
    print(songname)
    songid = df_songs['SongID'][count]
    print(songid)
    songid = validateTitle(songid)
    return songname, songid

In [7]:
# 取得動態歌詞
def get_lyrics(songid):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    url = 'http://music.163.com/api/song/lyric?id={}&lv=-1&kv=-1&tv=-1'.format(songid)
    response = requests.get(url, headers=headers)
    data = response.json()
    return data

In [8]:
# 下載動態歌詞
# 解決UnicodeDecodeError: 'cp950'問題，加入encoding="utf-8"
def save_lrc_file(data, songname, songid):
    content = data['lrc']['lyric']
    filepath = 'CloudMusic_lrc/'+ songname + '_' + songid +'.lrc'
    with open(filepath, 'w', encoding="utf-8") as file:
        file.write(content)
    print(filepath)

In [9]:
# 主程式抓歌曲
playlist_id = '2781412735'
[song_ids, song_titles, song_url, list_title, list_url] = get_songs(playlist_id)
df_songs = get_songinfo(song_ids, song_titles, song_url)
for i in range(len(df_songs)):
    songname, songid = get_name_artist(df_songs, i)
    try:
        data = get_lyrics(songid)
        save_lrc_file(data, songname, songid)
    except:
        pass
    print('-'*70)

Havana
494865824
CloudMusic_lrc/Havana_494865824.lrc
----------------------------------------------------------------------
Tip of Your Thumb
528271266
CloudMusic_lrc/Tip of Your Thumb_528271266.lrc
----------------------------------------------------------------------
Paris in the Rain
518904648
CloudMusic_lrc/Paris in the Rain_518904648.lrc
----------------------------------------------------------------------
Matches (Acoustic) [feat. Aaron Richards]
439911153
CloudMusic_lrc/Matches (Acoustic) [feat. Aaron Richards]_439911153.lrc
----------------------------------------------------------------------
All We Do 
32897951
CloudMusic_lrc/All We Do _32897951.lrc
----------------------------------------------------------------------
Million Dollar Mood
1337296869
CloudMusic_lrc/Million Dollar Mood_1337296869.lrc
----------------------------------------------------------------------
Last Call
1380834060
CloudMusic_lrc/Last Call_1380834060.lrc
-----------------------------------------------

CloudMusic_lrc/The Spectre_506092035.lrc
----------------------------------------------------------------------
Break Up in a Small Town
29592100
CloudMusic_lrc/Break Up in a Small Town_29592100.lrc
----------------------------------------------------------------------
Girl in the Mirror
1360909338
CloudMusic_lrc/Girl in the Mirror_1360909338.lrc
----------------------------------------------------------------------
Op Opa
1367034768
CloudMusic_lrc/Op Opa_1367034768.lrc
----------------------------------------------------------------------
degauss-Walking in the Sun（姚智鑫   IZZI remix）
1386522410
CloudMusic_lrc/degauss-Walking in the Sun（姚智鑫   IZZI remix）_1386522410.lrc
----------------------------------------------------------------------
Fresh Off The Grill
37091383
CloudMusic_lrc/Fresh Off The Grill_37091383.lrc
----------------------------------------------------------------------
Jesus In LA
1377700719
CloudMusic_lrc/Jesus In LA_1377700719.lrc
---------------------------------------

CloudMusic_lrc/All We Know_432698441.lrc
----------------------------------------------------------------------
Something Just Like This
461347998
CloudMusic_lrc/Something Just Like This_461347998.lrc
----------------------------------------------------------------------
Hope
1333199956
CloudMusic_lrc/Hope_1333199956.lrc
----------------------------------------------------------------------
Closer
423228325
CloudMusic_lrc/Closer_423228325.lrc
----------------------------------------------------------------------
Shape of You
451703096
CloudMusic_lrc/Shape of You_451703096.lrc
----------------------------------------------------------------------
Galway Girl
460043704
CloudMusic_lrc/Galway Girl_460043704.lrc
----------------------------------------------------------------------
Thinking Out Loud
28680438
CloudMusic_lrc/Thinking Out Loud_28680438.lrc
----------------------------------------------------------------------
Counting Stars
26060065
CloudMusic_lrc/Counting Stars_26060065.lrc
-

In [112]:
# 儲存音樂
def save_mp3_file(filename, content):
    filepath = 'CloudMusic_mp3/'+ filename
    with open(filepath, mode = "wb") as f:
        f.write(content)
    print(filepath)

In [10]:
# 下載音樂
# https://kknews.cc/zh-tw/comic/o26onj6.html
def get_mp3(songname, songid):
    url = 'http://music.163.com/song/media/outer/url?id=' + str(songid)+'.mp3'
    try:
        print("正在下載：{0}".format(songname))
        path = 'CloudMusic_mp3'
        urllib.request.urlretrieve(url, '{0}\\{1}_{2}.mp3'.format(path, songname, songid))
        print("Finish...") 
    except:
        print("Fail...")

In [115]:
# 下載音樂.mp3
def download_music(songname, songid):
    url = 'http://music.163.com/song/media/outer/url?id=' + str(songid)+'.mp3'
    response = requests.get(url)
    content = response.content
    # m4a檔案命名規則
    save_mp3_file(songname + '_' + songid + '.mp3', content)

In [116]:
download_music(songname, songid)

CloudMusic_mp3/Havana_494865824.m4a


In [159]:
# 版權問題要拿到真實的url才能下載
# 歌曲頁面選擇network，挑選All，選擇結尾為.m4a，取得Request URL即為下載連結
# https://kknews.cc/zh-tw/tech/ganvq2m.html
url = 'https://m801.music.126.net/20200112220148/ea1af8d09db0868d27634b29499b6dee/jdyyaac/520f/0158/0308/eefd2984f862ac2408b94bb0011ebcfa.m4a'
response = requests.get(url)
content = response.content
# m4a檔案命名規則
save_mp3_file('test.mp3', content)

CloudMusic_mp3/test.mp3


In [11]:
#取得歌曲的歌手名稱
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
url = 'https://music.163.com/#/playlist?id=2781412735'
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, "html.parser")
title = soup.select_one('.m-table')
print(title)

<table class="m-table {if type=='rank'}m-table-rank{/if}">
<thead>
<tr>
<th class="first {if type=='rank'}wrk{else}w1{/if}"><div class="wp"> </div></th>
<th><div class="wp af0"></div></th>
<th class="w2"><div class="wp af1"></div></th>
<th class="w3"><div class="wp af2"></div></th>
<th class="w4"><div class="wp af3"></div></th>
</tr>
</thead>
<tbody>
{list beg..end as y}
{var x=xlist[y]}
<tr class="{if y%2==0}even{/if} {if disable(x)}js-dis{/if}" id="${x.id|seed}">
<td class="left">
<div class="hd {if type=='rank'}rank{/if}">
<span class="ply {if isPlaying(x)}ply-z-slt{/if}" data-res-action="play" data-res-data="${from.fdata}" data-res-id="${x.id}" data-res-type="18" from}data-res-from="${from.fid}" if}="" {="" {if=""> </span>
<span class="num">${y+1}</span>
{if type=='rank'}
<div class="rk rk-1">
{if x.lastRank&gt;=0}
{if y-x.lastRank&gt;0}
<span class="ico u-icn u-icn-74 s-fc10">${y-x.lastRank}</span>
{elseif y-x.lastRank==0}
<span class="ico u-icn u-icn-72 s-fc4">0</span>
{else}
<sp

In [178]:
# https://stackoverflow.com/questions/52238679/how-to-find-header-data-and-name-python-requests/52239873
# https://stackabuse.com/the-python-requests-module/
r = requests.get(url)
r.headers['etag']

KeyError: 'etag'

In [ ]:
df_songs = get_songinfo(song_ids, song_titles, song_url)
# 取得SongURL
df_songs['SongURL'][10]